In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os

In [7]:
# 1. SETTING PATH
train_dir = "../car_damage/training"
val_dir = "../car_damage/validation"
# Parameter
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
# 2. LOAD DATASET
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)
class_names = train_ds.class_names
print(f"Kategori kerusakan: {class_names}")

Found 1383 files belonging to 3 classes.
Found 248 files belonging to 3 classes.
Kategori kerusakan: ['01-minor', '02-moderate', '03-severe']


In [8]:
# 2. LOAD MODEL LAMA
print("Memuat model lama untuk Fine-Tuning...")
model = tf.keras.models.load_model("car_damage_model.keras")
# 3. PROSES FINE-TUNING
# Kita cari lapisan 'mobilenetv2' di dalam model Sequential kita
base_model = model.get_layer("mobilenetv2_1.00_224") # Nama default MobileNetV2 di Keras
base_model.trainable = True
# Kunci lapisan bawah, buka 20 lapisan teratas
# Ini membuat AI bisa menyesuaikan detail tekstur (lecet/penyok)
for layer in base_model.layers[:-20]:
    layer.trainable = False
# 4. RE-COMPILE dengan Learning Rate super pelan (Cukup 10^-5)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Memuat model lama untuk Fine-Tuning...


In [9]:
# 5. LANJUT TRAINING
print("Memulai Fine-Tuning... fokus pada detail kerusakan.")
EPOCHS = 15 # Tidak perlu terlalu lama
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

Memulai Fine-Tuning... fokus pada detail kerusakan.
Epoch 1/15
44/44 ━━━━━━━━━━━━━━━━━━━━ 30s 471ms/step - accuracy: 0.5401 - loss: 0.9299 - val_accuracy: 0.7016 - val_loss: 0.6815
Epoch 2/15
44/44 ━━━━━━━━━━━━━━━━━━━━ 18s 419ms/step - accuracy: 0.6182 - loss: 0.8246 - val_accuracy: 0.6895 - val_loss: 0.6834
Epoch 3/15
44/44 ━━━━━━━━━━━━━━━━━━━━ 21s 488ms/step - accuracy: 0.6189 - loss: 0.8144 - val_accuracy: 0.6895 - val_loss: 0.6797
Epoch 4/15
44/44 ━━━━━━━━━━━━━━━━━━━━ 20s 450ms/step - accuracy: 0.6356 - loss: 0.7932 - val_accuracy: 0.6976 - val_loss: 0.6751
Epoch 5/15
44/44 ━━━━━━━━━━━━━━━━━━━━ 19s 427ms/step - accuracy: 0.6399 - loss: 0.7981 - val_accuracy: 0.7016 - val_loss: 0.6685
Epoch 6/15
44/44 ━━━━━━━━━━━━━━━━━━━━ 18s 417ms/step - accuracy: 0.6240 - loss: 0.7890 - val_accuracy: 0.7056 - val_loss: 0.6656
Epoch 7/15
44/44 ━━━━━━━━━━━━━━━━━━━━ 19s 419ms/step - accuracy: 0.6435 - loss: 0.7746 - val_accuracy: 0.7056 - val_loss: 0.6646
Epoch 8/15
44/44 ━━━━━━━━━━━━━━━━━━━━ 19s 419

In [ ]:
# 6. SIMPAN SEBAGAI VERSI TERBAIK
model.save("car_damage_model_v2.keras")
print("✅ Fine-Tuning Selesai! Model V2 tersimpan.")

Model Kerusakan berhasil disimpan: car_damage_model.keras
